<a href="https://colab.research.google.com/github/SusheelThapa/ML-From-Scratch/blob/tensorflow/tensorflow/tensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning Fundamentals

## Introduction of Tensorflow

## Installing Tensorflow

Use the below command, to install the ***tensorflow** in your local machine

```bash
pip install tensorflow
```

## Importing Tensorflow

In [ ]:
import tensorflow as tf
tf.version

## What is tensor?

Tensor is a generalization of vectors and matrices to potentially higher dimension.

Internally, tensorflow represent tensors as  n-dimensional arrays of base datatypes.

Each tensor has a data type and a shape

**Data Types** includes: float32, int32, string and others

**Shape**: Represents the dimension of data

### Creating tensor

Below are the examples of creating tensor

In [ ]:
string = tf.Variable("This is a string", tf.string)
number = tf.Variable(324, tf.int16)
floating = tf.Variable(3.567,tf.float64)

### Rank/Degree of Tensors

Another word for rank is degree, it can be define as the number of dimensions involved in the tensor.

In the above code block, what we have created is *tensor of rank zero*

Now, let's create tensor of higher degree/ranks

In [ ]:
rank1_tensor = tf.Variable(["Something","Nothing"], tf.string)

To find the rank of the tensor we can call `rank()` method as 

In [ ]:
tf.rank(rank1_tensor)

### Shape of Tensors

Shape of the tensors is simply the amount of elements that exist in each dimension.

*Tensorflow will try to determine the shape of a tensor but sometimes it may be unknown*

To get the shape of the tensor, we can call **shape attribute***

In [ ]:
rank1_tensor.shape

### Changing the shape

Number of elements of a tensor is the product of the sizes of all its shape.

Due to which many shapes that have the same number of elements, making it convient to be able to change the shape of a tensor

Example of changing the shape of tensor

In [ ]:
tensor1 = tf.ones([1,2,3]) # tf.ones will create tensor of provide shape will all its element of ones

tensor2 = tf.reshape(tensor1,[3,2,1]) # reshape the existing tensor to shape [3,2,1]

tensor3= tf.reshape(tensor2,[3,-1]) # -1 tells tensor to calculate the size of the dimension at that place

# The number of elements in orginal tensor and the reshape tensor is same

Now, lets have a look at the shape of the tensor we have created

In [ ]:
print(tensor1.shape)
print(tensor2.shape)
print(tensor3.shape)

### Types of tensor

Commonly used tensor are as follows:
- Variable
- Constant
- Placeholder
- SparseTensor